# 操作系统基本概念

## 为什么要有操作系统
如果没有操作系统，每个应用程序需要直接调用硬件设备，此时需要有OS lib作为统一的interface，让每个应用进程以单独调用lib的方式来使用硬件。就像python调用python库一样。\
<font color=green>**问题：无法在实现multiplexing的同时实现(memory) isolation**</font>
1. <font color=blue>无法实现multiplexing</font>: \
此时所有的硬件资源没有一个独立的第三方管理者。每个进程都调用自己的OS lib来与硬件交互，要实现multiplexing就需要应用程序自己的管理机制能够周期性地放弃硬件使用权，或者有一个app之间的cooprative schedule。但这会导致：\
(1) app忘记放弃硬件使用权，别的app无法运行 \
(2) app在放弃硬件使用权之前陷入死循环，导致别的app无法运行 \
(3) 当app的运行出现问题后，甚至无法通过别的app将其关闭
2. <font color=blue>无法实现memory isolation，这也会导致无法实现multiplexing</font>：\
即使通过cooperative schedule的方式实现了multiplexing，由于所有app共享物理内存，一个app可能覆盖其他app的memory，甚至覆盖掉操作系统的memory。要实现memory isolation，就要有一个分配管理内存的工具。

<font color=red>**所以，为了实现multiplexing，需要同时做到CPU isolation和memory isolation。此时，需要一个独立的第三方工具来管理CPU和memory等资源的分配。**</font>

## OS需要实现的功能目标
1. 便于用户使用硬件资源，运行不同的application。\
实现方式是提供统一的interface，让用户不用关心底层特征
2. 实现多应用的multiplex
3. 实现应用之间的isolation
4. 实现应用之间的share

## 实现功能目标的方式：Virtualization
1. Unix系统的设计思路：用interface来抽象硬件资源，并且抽象的方式非常便于实现multiplexing和memory isolation。
2. Unix中的典型抽象方式：\
(1) process <=> CPU: OS给进程分配CPU cores，同时也强制进程周期性地放弃CPU使用权。 \
(2) exec <=> memory: OS给进程分配memory，并实现不同进程之间的memory isolation。 \
(3) files <=> Disk blocks: 通过OS的read/write files interface实现在disk上的读写操作，OS还提供了name files的接口，以及在process和user之间共享files的接口。 \
(4) pipes <=> shared memory: 

## 操作系统的其他目标
1. performance
2. user-friendly (flexibility)
3. security

## OS实现安全性需要的硬件支持
1. 安全性的基本要求： \
(1) an application不能crash OS。crash OS会使得其他应用程序也无法正常工作。 \
(2) an application不能打破isolation。不然恶意进程就可以对系统发起攻击，比如恶意改变文件内容。 \
<font color=red>**因此，在app和OS之间，app之间都要有strong isolation。**</font>
2. **实现方式：靠硬件支持** \
所有支持multiplexing的处理器都有两类设计：user/kernel mode和Virtual memory。\
(1) <font color=blue>**user/kernel mode**</font> \
OS在kernel mode中运行，能执行privileged instructions。user app在user mode中运行，不能执行privileged instructions。由此控制应用程序能实现的功能范围。\
(2) <font color=blue>**virtual memory**</font> \
processor提供page table把物理内存映射到Virtual address上。OS设置page table来决定进程可以access哪些内存。由此实现memory isolation。